<a href="https://colab.research.google.com/github/Ziyi-star/Bachelor-Project/blob/main/notebooks/handlebar/LSTM/Lstm__Handlebar_best.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# For Vscode
import numpy as np
import pandas as pd
from datetime import datetime
from matplotlib import pyplot as plt
from matplotlib import dates as md
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
import seaborn as sns
RANDOM_SEED = 42

import sys
sys.path.append('../../../')   # Add parent directory to Python path
from utils.Plots import *

In [7]:
df = pd.read_csv('../../../data/handlebar/train_data.csv')
df

,curb_scene,Acc-Z_1,Acc-Z_2,Acc-Z_3,Acc-Z_4,Acc-Z_5,Acc-Z_6,Acc-Z_7,Acc-Z_8,Acc-Z_9,...,Acc-Z_91,Acc-Z_92,Acc-Z_93,Acc-Z_94,Acc-Z_95,Acc-Z_96,Acc-Z_97,Acc-Z_98,Acc-Z_99,Acc-Z_100
0,0.0,9.595963,11.133041,4.472366,4.592072,5.310333,17.501617,17.319656,11.291061,1.508347,...,12.737152,5.166687,1.039078,4.481949,12.052414,14.475342,10.692505,5.358231,9.576813,11.377243
1,0.0,10.783478,6.344635,7.445969,5.238510,13.091492,11.386826,10.783478,11.496963,6.526596,...,1.403000,5.516235,14.652512,20.005950,19.383453,8.058884,-1.537079,1.977615,7.565674,12.028473
2,0.0,3.337524,7.517792,9.902420,10.031708,8.283936,4.472366,3.035843,3.643982,7.814682,...,14.216766,7.493851,5.602432,4.893753,6.569687,9.054871,15.021225,13.584702,7.206543,3.801987
3,0.0,18.243820,16.591812,5.564117,0.545883,5.013458,14.211975,14.006073,8.705322,2.978394,...,11.310211,-10.534485,16.476898,21.538239,-7.268799,-34.404680,-28.218063,5.147537,11.401184,-16.716324
4,0.0,4.596863,6.186615,15.246277,13.919891,8.963898,2.796432,6.962341,13.867218,13.541611,...,14.494492,3.821152,-0.531509,0.780502,14.202408,24.056946,18.804062,7.503433,4.218582,7.766785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7448,0.0,11.846512,12.756302,10.113113,3.821152,2.154785,4.836289,-0.900223,8.734055,6.368576,...,7.220917,8.556885,5.674255,13.450623,12.866440,11.535263,9.323029,9.375702,12.971787,10.429138
7449,0.0,8.657440,8.633499,8.652649,8.609543,8.633499,8.638275,8.647858,8.623917,8.619125,...,8.643066,8.647858,8.643066,8.633499,8.647858,8.643066,8.633499,8.619125,8.647858,8.633499
7450,0.0,8.643066,8.647858,8.633499,8.638275,8.643066,8.638275,8.633499,8.638275,8.643066,...,8.643066,8.623917,8.652649,8.633499,8.633499,8.652649,8.647858,8.652649,8.633499,8.647858
7451,0.0,8.015793,7.814682,4.362228,8.906433,12.660538,13.761871,10.079590,8.599976,5.415680,...,11.520905,22.045807,16.041153,6.971909,-0.167587,5.171478,11.343735,21.327545,9.629486,-5.104431


In [8]:
#scaling data, because neural network works better with scaled data
scaler = StandardScaler()
data_scaled = scaler.fit(df.iloc[:, 1:])
#transform data
train_data_scaled = scaler.transform(df.iloc[:, 1:])


In [9]:
# Convert the scaled data back to DataFrames for further processing
train_data_scaled_df = pd.DataFrame(train_data_scaled, columns=df.columns[1:])
train_data_scaled_df['curb_scene'] = df['curb_scene'].values
train_data_scaled_df.shape

(7453, 101)

In [11]:
# Filter and slice the data
normal_train_data_scaled = train_data_scaled_df[train_data_scaled_df['curb_scene'] == 0.0].iloc[:, :-1]
abnormal_train_data_scaled = train_data_scaled_df[train_data_scaled_df['curb_scene'] == 1.0].iloc[:, :-1]

In [12]:
# Cast to float32
normal_train_data_scaled = normal_train_data_scaled.astype("float32")
abnormal_train_data_scaled = abnormal_train_data_scaled.astype("float32")
normal_train_data_scaled.shape, abnormal_train_data_scaled.shape

((7178, 100), (275, 100))

In [10]:
# Initialize and train One-Class SVM, Using only normal data for training
ocsvm = OneClassSVM(kernel='rbf', nu=0.001, gamma='scale')
ocsvm.fit(normal_train_data_scaled)

OneClassSVM(nu=0.001)

In [10]:
import joblib

# Save the model
joblib.dump(ocsvm, '../../../models/ocsvm_handlebar.joblib')

['../../../models/ocsvm_handlebar.joblib']

In [40]:
# GridSearchCV
grid = GridSearchCV(
    OneClassSVM(), 
    param_grid, 
    scoring=make_scorer(custom_scorer, greater_is_better=True), 
    cv=5,
    verbose=1,
    n_jobs=-1
)
grid.fit(X_train, np.zeros(len(X_train)))  # Dummy labels for training

Fitting 5 folds for each of 60 candidates, totalling 300 fits


c:\Users\liuzi\miniconda3\Lib\site-packages\sklearn\model_selection\_search.py:1108: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
  warnings.warn(


GridSearchCV(cv=5, estimator=OneClassSVM(), n_jobs=-1,
             param_grid={'gamma': ['scale', 'auto', 0.001, 0.01, 0.1],
                         'kernel': ['rbf', 'linear', 'sigmoid', 'poly'],
                         'nu': [0.001, 0.005, 0.01]},
             scoring=make_scorer(custom_scorer, response_method='predict'),
             verbose=1)

In [41]:
# Best parameters
print(f"Best parameters: {grid.best_params_}")
best_ocsvm = grid.best_estimator_

Best parameters: {'gamma': 'scale', 'kernel': 'rbf', 'nu': 0.001}
